# 🚀  How to Build & Test Your Streamlit App in Colab
This notebook is a template for developing and testing your Streamlit app. You will use this to write your code, test it live, and then download the final files to upload to GitHub.



<img src="https://raw.githubusercontent.com/sasrwt/homequity/main/streamlitappflow.png" width="500">

### **Step 1: Install All Libraries**
Run the code cell below. This installs Streamlit, the libraries your app needs (pandas, scikit-learn), and localtunnel (creates a temporary public URL that "tunnels" to the hmeq app running insidColab notebook. This makes our app accessible in any web browser.).

In [1]:
# Install all required libraries
!pip install streamlit pandas scikit-learn -q
!npm install localtunnel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.5 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

### **Step 2: Write Your App Code (`app.py`)**

https://docs.streamlit.io/develop/api-reference/widgets

Run the cell below.  
The `%%writefile` command will take all the code in the cell and save it as a new file named **`app.py`**.

This is where you will do your main work. You need to:

← **Update the `st.slider` and `st.selectbox` widgets** to match the input columns your model requires.

← **Update the `pd.DataFrame`** so its columns match the widgets you created.

← **Change the file path** so it correctly points to the location of your uploaded **`.pkl`** model file.


In [ ]:
%%writefile hmeqapp.py

# -*- coding: utf-8 -*-
import streamlit as st
import pickle
import pandas as pd
import sklearn  # This is needed for the pickle file to load!

# Load the trained model
# --- Put the Model in Drive First---
with open("/content/sample_data/hmeq_model.pkl", "rb") as file:
    model = pickle.load(file)

# Title for the app
# st.title("Home Equity Loan Approval")
st.markdown(
    "<h1 style='text-align: center; background-color: #ffcccc; padding: 10px; color: #cc0000;'><b>Home Equity Loan Approval</b></h1>",
    unsafe_allow_html=True
)

# Numeric inputs
st.header("Enter Loan Applicant's Details")

# Input fields for numeric values
loan = st.slider("Loan Amount (LOAN)", min_value=1000, max_value=500000, step=1000)
mortdue = st.slider("Mortgage Due (MORTDUE)", min_value=0.0, max_value=1000000.0, step=1000.0)
value = st.slider("Property Value (VALUE)", min_value=0.0, max_value=1000000.0, step=1000.0)
yoj = st.selectbox("Years at Job (YOJ)", options=list(range(1, 41)))  # Options from 1 to 40
derog = st.number_input("Derogatory Reports (DEROG)", min_value=0, max_value=15, step=1)
delinq = st.selectbox("Delinquent Reports (DELINQ)", options=list(range(0, 15)))  # Options from 0 to 10
clage = st.slider("Age of Oldest Trade Line in Months (CLAGE)", min_value=0.0, max_value=100.0, step=1.0)
ninq = st.slider("Number of Recent Credit Inquiries (NINQ)", min_value=0.0, max_value=15.0, step=1.0)
clno = st.slider("Number of Credit Lines (CLNO)", min_value=0.0, max_value=50.0, step=1.0)
debtinc = st.slider("Debt-to-Income Ratio (DEBTINC)", min_value=0.0, max_value=200.0, step=0.1)

# Categorical inputs with options
reason = st.selectbox("Reason for Loan (REASON)", ["HomeImp", "DebtCon"])
job = st.selectbox("Job Category (JOB)", ["ProfExe", "Other", "Mgr", "Office", "Sales"])

# Create the input data as a DataFrame
input_data = pd.DataFrame({
    "LOAN": [loan],
    "MORTDUE": [mortdue],
    "VALUE": [value],
    "YOJ": [yoj],
    "DEROG": [derog],
    "DELINQ": [delinq],
    "CLAGE": [clage],
    "NINQ": [ninq],
    "CLNO": [clno],
    "DEBTINC": [debtinc],
    "REASON": [reason],
    "JOB": [job]
})

# --- Prepare Data for Prediction ---
# 1. One-hot encode the user's input.
input_data_encoded = pd.get_dummies(input_data, columns=['REASON', 'JOB'])

# 2. Add any "missing" columns the model expects (fill with 0).
model_columns = model.feature_names_in_
for col in model_columns:
    if col not in input_data_encoded.columns:
        input_data_encoded[col] = 0

# 3. Reorder/filter columns to exactly match the model's training data.
input_data_encoded = input_data_encoded[model_columns]

# Predict button
if st.button("Evaluate Loan"):
    # Predict using the loaded model
    prediction = model.predict(input_data_encoded)[0]

    # Display result
    if prediction == 1:
        st.write("The prediction is: **Bad Loan** 🚫")
    else:
        st.write("The prediction is: **Good Loan** 💲")



        """
What happens if the user enters a value not in the training data?

Example: User enters REASON = 'Vacation', but the model only knows 'DebtCon' and 'HomeImp'.

1. pd.get_dummies creates a new column: REASON_Vacation = 1.
2. The code then adds the *known* columns: REASON_DebtCon = 0 and REASON_HomeImp = 0.
3. The final filtering step *drops* the unknown REASON_Vacation column because it's not in the
   model's expected feature list.

Result: The model receives REASON_DebtCon = 0 and REASON_HomeImp = 0, which correctly
treats the unknown 'Vacation' input as "none of the known categories" (i.e., "Other").
"""

Writing hmeqapp.py


### **Step 3: Write Your Requirements File**

Need this file in GitHub.  
It tells **Streamlit Cloud** which libraries need to be installed.

Run the cell below to create **`requirements.txt`**.

← **Important:** If your model uses additional libraries (such as `xgboost`, `lightgbm`, `catboost`, etc.),  
you **must manually add them** to the list before deploying!


In [2]:
%%writefile requirements.txt
streamlit
pandas
scikit-learn
numpy

Writing requirements.txt


### **Step 4: Test Your App Live!**

Run the code cell below.  
It will start your **Streamlit app** in the background and then give you a **public URL** to test it.

← Click the URL (it will look something like `https://some-random-words.loca.lt`).

← When the new page opens, click the **"Click to Continue"** button.

← Your app will appear! **Test it thoroughly** to make sure all inputs, predictions, and outputs work as expected.


In [ ]:
# Run the app in the background
!streamlit run hmeqapp.py.py &>/dev/null&

# Get the password (your IP) and print it.
# The "&" runs it in the background while the tunnel starts.
print("Fetching your tunnel password...")
!curl https://loca.lt/mytunnelpassword &

# Start the tunnel. This command will keep the cell running and give you the URL.
!npx localtunnel --port 8501

Fetching your tunnel password...
34.16.156.14⠙your url is: https://light-mice-train.loca.lt


## Kick the Can and Move On


That TypeError: Failed to fetch is a browser-side error. It means your browser (the app's frontend) tried to download its own JavaScript files from the localtunnel URL, but the tunnel was so slow or unstable that the request failed.

This confirms our theory: your app is taking too long to load the model, and the localtunnel connection is collapsing or timing out before the app can become stable.

💡 The Real Problem
The "Colab + localtunnel" workflow is just a testing tool. It is famously unstable and difficult, as you've seen.

We have been using this flaky, complicated method (managing passwords, running multiple cells, curl commands) just to test the app. But this test is now causing more problems than it's worth.

The good news is we don't need it anymore. We have already learned everything we need from this Colab experiment:

We have the final, working hmeqapp.py code.

We have the final, working requirements.txt file (after fixing the sklearn typo).

We know the hmeq_model.pkl file works.

The goal was never to use the Colab app. The goal was to get the files ready for a real deployment. That mission is accomplished.

### **Step 6: Download Your Files for GitHub**

If your app works, you're ready for deployment.

← Go to the **Files** sidebar on the left.

← Find **`app.py`**. Right-click it and select **Download**.

← Find **`requirements.txt`**. Right-click it and select **Download**.

You are now ready to upload **`app.py`**, **`requirements.txt`**, and your **`.pkl` model file** to a new GitHub repository for deployment.

---

### **Final GitHub Step (Important!)**

Before deploying, edit your **`app.py`** file **one last time** (either in GitHub or in a local editor):

← **Remove the folder path** from your `open()` command.  
It should look like this:

```python
open("your_model_name.pkl", "rb")
